In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def gen_xn(N, d=2):
    return np.random.uniform(-1,1,(N,d))


In [3]:
def gen_f(xn, d=2):
    x1=np.random.uniform(-1,1,d)
    x2=np.random.uniform(-1,1,d)

    m= (x1[1]-x2[1])/(x1[0]-x2[0])
    c=x1[1]-m*x1[0]

    y_f=[]
    for i,x in enumerate(xn):
        if x[1] > x[0]*m+c:
            y_f.append(1)
        else: 
            y_f.append(-1)

    out=np.concatenate((xn, np.array(y_f).reshape(-1, 1)), axis=-1)

    return out, [m,c]
    

In [4]:
def sign(X):
    out=[]

    for x in X:
        if x>0:
            out.append(1)
        elif x<0:
            out.append(-1)
        elif x==0:
            out.append(0)
    return out

In [5]:
def PLA(i_p):
    w=np.zeros(3)
    x,y= i_p[:, :2], i_p[:, -1]    
    x=np.concatenate((x, np.ones((x.shape[0], 1))), axis=-1)
    i=0
    while(True):
        i+=1
        h=sign(np.sum(w*x, axis=1))
        score=np.sum(abs(h-y))
        idx=np.argwhere((h-y)!=0)
        #print('predicted:', h ,'\n true:', y)
        #print('score', score)
        if score ==0: break
        idx=idx[np.random.randint(idx.shape[0])]
        
        w=w+y[idx]*x[idx].reshape(w.shape)

    return w, i
    

In [6]:
def plot(w_t, w_p, out_f):
    m_t=w_t[0]
    c_t=w_t[1]
    

    y= [-1,1]
    x_t= [(yi-c_t)/m_t for yi in y]
    x_p= [np.sum(w_p[1:]*np.array([yi,1]))/-w_p[0] for yi in y]
    plt.plot(x_t, y, color='g')
    plt.plot(x_p, y, color='b')
    
    pos=out_f[np.where(out_f[:,-1]>0)]
    neg=out_f[np.where(out_f[:,-1]<0)]
 
    try:
        plt.scatter(pos[:,0], pos[:,1], s=50, color='g', marker='+')
    except IndexError:
        pass
    try: 
        plt.scatter(neg[:,0], neg[:,1], s=50, color='r', marker='_')
    except IndexError:
        pass


In [7]:
it=[]
p_e=[]
N=100
for i in range(1000):
    data=gen_xn(N)
    f, w_t=gen_f(data)
    w_p,i=PLA(f)
    it.append(i)
    m,c=w_t
    test_data=np.concatenate((gen_xn(N), np.ones((N, 1))), axis=-1)
    f_x= sign(np.sum([m,-1,c]*test_data, axis=1))
    g_x= sign(np.sum(w_p*test_data, axis=1))
    
    p_e.append(np.sum(int(f_x != g_x)) / N)
    #plot(w_t, w_p, f)
    #plt.figure()
print('Converged after:', np.mean(it))
print('Error percentage:', np.mean(p_e))



Converged after: 100.863
Error percentage: 0.01
